# Data Validation Notebook

This notebook performs data quality validation on processed data.

## Parameters
- `execution_date`: Date of pipeline execution
- `s3_bucket`: S3 bucket containing processed data
- `aws_endpoint_url`: AWS endpoint URL (LocalStack)

In [ ]:
# Parameters - these will be injected by Papermill
execution_date = '2024-01-01'
s3_bucket = 'processed-data'
aws_endpoint_url = 'http://localhost:4566'

In [ ]:
import pandas as pd
import boto3
import numpy as np
from datetime import datetime
import os
import json

# Configure AWS credentials for LocalStack
os.environ['AWS_ACCESS_KEY_ID'] = 'test'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'test'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

print(f"Execution Date: {execution_date}")
print(f"S3 Bucket: {s3_bucket}")
print(f"AWS Endpoint: {aws_endpoint_url}")

In [ ]:
# Initialize S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=aws_endpoint_url,
    aws_access_key_id='test',
    aws_secret_access_key='test',
    region_name='us-east-1'
)

# List objects in the processed data bucket
try:
    response = s3_client.list_objects_v2(Bucket=s3_bucket)
    if 'Contents' in response:
        print(f"Found {len(response['Contents'])} objects in bucket:")
        for obj in response['Contents'][:10]:  # Show first 10
            print(f"  - {obj['Key']} ({obj['Size']} bytes)")
    else:
        print("No objects found in bucket")
except Exception as e:
    print(f"Error listing bucket contents: {e}")

In [ ]:
# Define data quality rules
data_quality_rules = {
    'completeness': {
        'description': 'Check for missing values',
        'threshold': 0.05  # Max 5% missing values allowed
    },
    'uniqueness': {
        'description': 'Check for duplicate records',
        'threshold': 0.01  # Max 1% duplicates allowed
    },
    'validity': {
        'description': 'Check data format and ranges',
        'rules': {
            'amount': {'min': 0, 'max': 10000},
            'customer_id': {'min': 1}
        }
    },
    'consistency': {
        'description': 'Check referential integrity',
        'rules': {
            'category': ['Electronics', 'Clothing', 'Food', 'Books', 'Other'],
            'payment_method': ['Credit Card', 'Debit Card', 'Cash', 'PayPal']
        }
    }
}

print("Data Quality Rules:")
for rule_name, rule_config in data_quality_rules.items():
    print(f"  {rule_name}: {rule_config['description']}")

In [ ]:
# For demonstration, let's create some sample processed data
# In a real scenario, this would be read from S3
np.random.seed(42)
n_records = 9500  # Simulating some data loss during processing

processed_data = pd.DataFrame({
    'customer_id': range(1, n_records + 1),
    'transaction_date': pd.date_range(
        start=execution_date, 
        periods=n_records, 
        freq='min'
    ),
    'amount': np.random.normal(100, 25, n_records),
    'category': np.random.choice(
        ['Electronics', 'Clothing', 'Food', 'Books', 'Other'], 
        n_records
    ),
    'payment_method': np.random.choice(
        ['Credit Card', 'Debit Card', 'Cash', 'PayPal'], 
        n_records
    )
})

# Add some data quality issues for testing
processed_data.loc[0:10, 'amount'] = np.nan  # Missing values
processed_data = pd.concat([processed_data, processed_data.iloc[0:5]])  # Duplicates
processed_data.loc[20, 'amount'] = -100  # Invalid amount
processed_data.loc[21, 'category'] = 'InvalidCategory'  # Invalid category

print(f"Loaded {len(processed_data)} records for validation")
print(processed_data.head())

In [ ]:
# Validation Results Storage
validation_results = {
    'execution_date': execution_date,
    'timestamp': datetime.now().isoformat(),
    'total_records': len(processed_data),
    'tests': {}
}

# Test 1: Completeness
print("=== Completeness Test ===")
missing_data = processed_data.isnull().sum()
missing_percentage = (missing_data / len(processed_data)) * 100

completeness_passed = True
for column, missing_pct in missing_percentage.items():
    threshold_pct = data_quality_rules['completeness']['threshold'] * 100
    status = "PASS" if missing_pct <= threshold_pct else "FAIL"
    if status == "FAIL":
        completeness_passed = False
    print(f"  {column}: {missing_pct:.2f}% missing ({status})")

validation_results['tests']['completeness'] = {
    'passed': completeness_passed,
    'missing_data': missing_data.to_dict(),
    'missing_percentage': missing_percentage.to_dict()
}

print(f"\nCompleteness Test: {'PASSED' if completeness_passed else 'FAILED'}")

In [ ]:
# Test 2: Uniqueness
print("\n=== Uniqueness Test ===")
total_records = len(processed_data)
unique_records = len(processed_data.drop_duplicates())
duplicate_count = total_records - unique_records
duplicate_percentage = (duplicate_count / total_records) * 100

threshold_pct = data_quality_rules['uniqueness']['threshold'] * 100
uniqueness_passed = duplicate_percentage <= threshold_pct

print(f"  Total records: {total_records}")
print(f"  Unique records: {unique_records}")
print(f"  Duplicates: {duplicate_count} ({duplicate_percentage:.2f}%)")
print(f"  Status: {'PASS' if uniqueness_passed else 'FAIL'}")

validation_results['tests']['uniqueness'] = {
    'passed': uniqueness_passed,
    'total_records': total_records,
    'unique_records': unique_records,
    'duplicate_count': duplicate_count,
    'duplicate_percentage': duplicate_percentage
}

print(f"\nUniqueness Test: {'PASSED' if uniqueness_passed else 'FAILED'}")

In [ ]:
# Test 3: Validity (Range checks)
print("\n=== Validity Test ===")
validity_results = {}
validity_passed = True

# Check amount ranges
amount_rules = data_quality_rules['validity']['rules']['amount']
invalid_amounts = processed_data[
    (processed_data['amount'] < amount_rules['min']) | 
    (processed_data['amount'] > amount_rules['max'])
]
amount_valid = len(invalid_amounts) == 0
if not amount_valid:
    validity_passed = False

print(f"  Amount range check: {'PASS' if amount_valid else 'FAIL'}")
if not amount_valid:
    print(f"    Invalid amounts found: {len(invalid_amounts)}")

validity_results['amount_range'] = {
    'passed': amount_valid,
    'invalid_count': len(invalid_amounts)
}

# Check customer_id
customer_id_rules = data_quality_rules['validity']['rules']['customer_id']
invalid_customer_ids = processed_data[
    processed_data['customer_id'] < customer_id_rules['min']
]
customer_id_valid = len(invalid_customer_ids) == 0
if not customer_id_valid:
    validity_passed = False

print(f"  Customer ID check: {'PASS' if customer_id_valid else 'FAIL'}")

validity_results['customer_id'] = {
    'passed': customer_id_valid,
    'invalid_count': len(invalid_customer_ids)
}

validation_results['tests']['validity'] = {
    'passed': validity_passed,
    'details': validity_results
}

print(f"\nValidity Test: {'PASSED' if validity_passed else 'FAILED'}")

In [ ]:
# Test 4: Consistency (Categorical values)
print("\n=== Consistency Test ===")
consistency_results = {}
consistency_passed = True

# Check category values
valid_categories = data_quality_rules['consistency']['rules']['category']
invalid_categories = processed_data[
    ~processed_data['category'].isin(valid_categories)
]
category_consistent = len(invalid_categories) == 0
if not category_consistent:
    consistency_passed = False

print(f"  Category consistency: {'PASS' if category_consistent else 'FAIL'}")
if not category_consistent:
    print(f"    Invalid categories: {invalid_categories['category'].unique()}")

consistency_results['category'] = {
    'passed': category_consistent,
    'invalid_count': len(invalid_categories)
}

# Check payment method values
valid_payment_methods = data_quality_rules['consistency']['rules']['payment_method']
invalid_payment_methods = processed_data[
    ~processed_data['payment_method'].isin(valid_payment_methods)
]
payment_method_consistent = len(invalid_payment_methods) == 0
if not payment_method_consistent:
    consistency_passed = False

print(f"  Payment method consistency: {'PASS' if payment_method_consistent else 'FAIL'}")

consistency_results['payment_method'] = {
    'passed': payment_method_consistent,
    'invalid_count': len(invalid_payment_methods)
}

validation_results['tests']['consistency'] = {
    'passed': consistency_passed,
    'details': consistency_results
}

print(f"\nConsistency Test: {'PASSED' if consistency_passed else 'FAILED'}")

In [ ]:
# Overall validation summary
all_tests = validation_results['tests']
overall_passed = all(test['passed'] for test in all_tests.values())

validation_results['overall_status'] = 'PASSED' if overall_passed else 'FAILED'
validation_results['tests_summary'] = {
    'total_tests': len(all_tests),
    'passed_tests': sum(1 for test in all_tests.values() if test['passed']),
    'failed_tests': sum(1 for test in all_tests.values() if not test['passed'])
}

print("\n" + "="*50)
print("DATA QUALITY VALIDATION SUMMARY")
print("="*50)
print(f"Execution Date: {execution_date}")
print(f"Total Records: {validation_results['total_records']}")
print(f"Overall Status: {validation_results['overall_status']}")
print(f"Tests Passed: {validation_results['tests_summary']['passed_tests']}/{validation_results['tests_summary']['total_tests']}")
print()

for test_name, test_result in all_tests.items():
    status = "✅ PASSED" if test_result['passed'] else "❌ FAILED"
    print(f"  {test_name.title()}: {status}")

print("\n" + "="*50)

In [ ]:
# Save validation results
results_json = json.dumps(validation_results, indent=2)
results_filename = f"validation_results_{execution_date.replace('-', '_')}.json"
results_path = f"/tmp/{results_filename}"

with open(results_path, 'w') as f:
    f.write(results_json)

print(f"Validation results saved to: {results_path}")

# Upload results to S3
try:
    s3_key = f"validation-results/{execution_date}/{results_filename}"
    s3_client.upload_file(results_path, s3_bucket, s3_key)
    print(f"Results uploaded to S3: s3://{s3_bucket}/{s3_key}")
except Exception as e:
    print(f"Error uploading results to S3: {e}")

## Validation Complete

The data quality validation has been completed. The results have been saved and uploaded to S3.

### Key Findings:
- **Data Volume**: Processing completed with minimal data loss
- **Missing Values**: Some missing values detected (expected for raw data)
- **Duplicates**: A few duplicate records found (may need deduplication)
- **Invalid Values**: Some outliers detected (may need further investigation)
- **Categorical Integrity**: Most categorical values are valid

### Recommendations:
1. Implement automated alerts for failed validations
2. Add data profiling for statistical anomaly detection
3. Consider implementing data lineage tracking
4. Set up monitoring dashboards for data quality metrics